In [ ]:
import gymnasium as gym
import os, sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from models.dqn import DQN_Agent
import torch

In [ ]:
train_env = gym.make("ALE/Pong-ram-v5", render_mode=None)
test_env = gym.make("ALE/Pong-ram-v5", render_mode="human")

In [ ]:
model = DQN_Agent(n_observations=128, n_actions=6, big_net=True,
                  double_network=True, tau=1e-2, batch_size=128, lr=1e-4, eps_decay=0.9999)

In [ ]:
def train_model(model, n_games):
    for game in range(n_games):
        done = False
        obs, _ = train_env.reset()
        sum_reward = 0
        episode_length=0

        while not done:
            action = model.learning_act(obs)
            new_obs, reward, done,_, _ = train_env.step(action)
            model.store_memory(obs, new_obs, action, reward, done)

            model.learn()
            obs = new_obs

            sum_reward += reward
            episode_length += 1

        print(game, ": ", sum_reward)

In [ ]:
def evaluate_model(model, n_games):
    for _ in range(n_games):
        done = False
        observation, _ = test_env.reset()
        sum_reward = 0
        while not done:            
            action = model.act(observation)
            print(action)
            observation, reward, done, _, _ = test_env.step(action)
            sum_reward += reward
        print(f"{n_games}th game: {sum_reward}")

In [ ]:
train_model(model=model, n_games=100)
torch.save(model.policy_network.state_dict(), "../weights/pong_ram.pt")

In [ ]:
evaluate_model(model=model, n_games=5)